In [1]:
!pip install kaggle
!pip install kagglehub
!pip install nltk
nltk.download('stopwords')

NameError: name 'nltk' is not defined

In [2]:
import kagglehub
import pandas as pd
import os


C:\Users\sgaui\AppData\Local\Programs\Python\Python314\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")
print("Dataset stored at:", path)

Dataset stored at: C:\Users\sgaui\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1


In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sgaui\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
movie_data = pd.read_csv( r'C:\Users\sgaui\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1\IMDB Dataset.csv')

In [7]:
movie_data.shape

(50000, 2)

In [8]:
movie_data.head()

review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive

In [9]:
movie_data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
movie_data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [11]:
movie_data.replace({'sentiment':{'positive':5}}, inplace=True)

review sentiment
0      One of the other reviewers has mentioned that ...         5
1      A wonderful little production. <br /><br />The...         5
2      I thought this was a wonderful way to spend ti...         5
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...         5
...                                                  ...       ...
49995  I thought this movie did a down right good job...         5
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]

In [12]:
movie_data.replace({'sentiment':{'negative':1}}, inplace=True)

review sentiment
0      One of the other reviewers has mentioned that ...         5
1      A wonderful little production. <br /><br />The...         5
2      I thought this was a wonderful way to spend ti...         5
3      Basically there's a family where a little boy ...         1
4      Petter Mattei's "Love in the Time of Money" is...         5
...                                                  ...       ...
49995  I thought this movie did a down right good job...         5
49996  Bad plot, bad dialogue, bad acting, idiotic di...         1
49997  I am a Catholic taught in parochial elementary...         1
49998  I'm going to have to disagree with the previou...         1
49999  No one expects the Star Trek movies to be high...         1

[50000 rows x 2 columns]

In [13]:
print(movie_data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...         5
1      A wonderful little production. <br /><br />The...         5
2      I thought this was a wonderful way to spend ti...         5
3      Basically there's a family where a little boy ...         1
4      Petter Mattei's "Love in the Time of Money" is...         5
...                                                  ...       ...
49995  I thought this movie did a down right good job...         5
49996  Bad plot, bad dialogue, bad acting, idiotic di...         1
49997  I am a Catholic taught in parochial elementary...         1
49998  I'm going to have to disagree with the previou...         1
49999  No one expects the Star Trek movies to be high...         1

[50000 rows x 2 columns]


In [14]:
movie_data['sentiment'].value_counts()


sentiment
5    25000
1    25000
Name: count, dtype: int64

In [15]:

positive_reviews = movie_data[movie_data['sentiment'] == 5]
negative_reviews = movie_data[movie_data['sentiment'] == 1]


positive_sample = positive_reviews.sample(n=2000, random_state=42)
negative_sample = negative_reviews.sample(n=2000, random_state=42)


movie_data = pd.concat([positive_sample, negative_sample])


In [16]:
stem = PorterStemmer()

In [17]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [18]:
movie_data = movie_data.reset_index(drop=True)
movie_data.insert(0, 'sl_no', range(1, len(movie_data) + 1))



In [19]:
print(movie_data)


      sl_no                                             review sentiment
0         1  I don't know how or why this film has a meager...         5
1         2  For a long time it seemed like all the good Ca...         5
2         3  Terry Gilliam's and David Peoples' teamed up t...         5
3         4  What is there to say about an anti-establishme...         5
4         5  This movie was made only 48 years after the en...         5
...     ...                                                ...       ...
3995   3996  A genuinely odd, surreal jumble of visual idea...         1
3996   3997  As there was nothing wrong with the acting etc...         1
3997   3998  There are no spoilers here... Because there is...         1
3998   3999  This is almost Ed Wood territory. Yeah, that r...         1
3999   4000  For the most part, I considered this movie unw...         1

[4000 rows x 3 columns]


In [20]:
movie_data['stemmed_content'] = movie_data['review'].apply(stemming)

In [21]:
movie_data.head()

sl_no                                             review sentiment  \
0      1  I don't know how or why this film has a meager...         5   
1      2  For a long time it seemed like all the good Ca...         5   
2      3  Terry Gilliam's and David Peoples' teamed up t...         5   
3      4  What is there to say about an anti-establishme...         5   
4      5  This movie was made only 48 years after the en...         5   

                                     stemmed_content  
0  know film meager rate imdb film accompani curi...  
1  long time seem like good canadian actor head s...  
2  terri gilliam david peopl team creat one intel...  
3  say anti establish film produc time colourless...  
4  movi made year end civil war like anticip th a...

In [22]:
x=movie_data["stemmed_content"]
y=movie_data["sentiment"]

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, stratify=y, random_state=42)

In [24]:
print(x.shape,x_train.shape,x_test.shape)

(4000,) (3200,) (800,)


In [25]:
print(y.shape,y_train.shape,y_test.shape)

(4000,) (3200,) (800,)


In [26]:
vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=8000,
    min_df=2
)

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)


In [27]:
print(x_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 316679 stored elements and shape (3200, 8000)>
  Coords	Values
  (0, 4225)	0.0955745670388502
  (0, 6290)	0.07190430930623368
  (0, 5097)	0.21656858612212312
  (0, 643)	0.05609820718416897
  (0, 1286)	0.13995036013596812
  (0, 6254)	0.1169719989697749
  (0, 7970)	0.056740966500096274
  (0, 7047)	0.08180498926117995
  (0, 3940)	0.06421186029658751
  (0, 6909)	0.18308067771509215
  (0, 5483)	0.07376001732299237
  (0, 2528)	0.15860240115800434
  (0, 6759)	0.04798668847586639
  (0, 5472)	0.044465170862885046
  (0, 5672)	0.1073777387993312
  (0, 6695)	0.10569323304239801
  (0, 4140)	0.18455371862505168
  (0, 26)	0.09748712671168987
  (0, 440)	0.08378301904285312
  (0, 609)	0.04612455375381013
  (0, 5431)	0.06405883827349819
  (0, 5620)	0.08627643277302105
  (0, 7950)	0.0899505529889253
  (0, 769)	0.055540052454302505
  (0, 7083)	0.08201415067581044
  :	:
  (3199, 2799)	0.07628200764600342
  (3199, 4184)	0.1676697828005272
  (3199

In [28]:
print(x_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 77310 stored elements and shape (800, 8000)>
  Coords	Values
  (0, 79)	0.06931872743219303
  (0, 128)	0.12371047305242702
  (0, 146)	0.1044802192383546
  (0, 442)	0.08631710324044611
  (0, 762)	0.1205122326730759
  (0, 763)	0.15057766910171325
  (0, 769)	0.1688303920436365
  (0, 788)	0.08444491947473558
  (0, 856)	0.1675688505422345
  (0, 942)	0.1675688505422345
  (0, 1402)	0.08814303226258416
  (0, 1642)	0.14141950483224888
  (0, 1701)	0.12220938033338499
  (0, 1808)	0.12433521213770997
  (0, 1811)	0.11170970679908368
  (0, 1853)	0.0905476781193163
  (0, 1922)	0.18182288902138985
  (0, 2090)	0.08670362170586544
  (0, 2109)	0.109803760832793
  (0, 2167)	0.07817037972824152
  (0, 2219)	0.11865890684092527
  (0, 2249)	0.05550710999516221
  (0, 2260)	0.17477511063079537
  (0, 2468)	0.10854575177204326
  (0, 2896)	0.0905476781193163
  :	:
  (799, 3416)	0.09641132538412696
  (799, 4128)	0.1489519302015615
  (799, 4154)	0.05808700

In [29]:
rating_model = LogisticRegression(C=0.5, max_iter=1000)

In [30]:
print(y_train)

1719    5
2358    1
2490    1
1844    5
3427    1
       ..
765     5
554     5
1490    5
572     5
639     5
Name: sentiment, Length: 3200, dtype: object


In [31]:
print(type(y_train))
print(y_train.shape)
print(y_train.unique())
print(y_train.dtype)
print(y_train.isnull().sum())


<class 'pandas.Series'>
(3200,)
[5 1]
object
0


In [32]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [33]:
print(y_train)

1719    5
2358    1
2490    1
1844    5
3427    1
       ..
765     5
554     5
1490    5
572     5
639     5
Name: sentiment, Length: 3200, dtype: int64


In [34]:
rating_model.fit(x_train,y_train)

LogisticRegression(C=0.5, max_iter=1000)

In [35]:
x_train_prediction = rating_model.predict(x_train)

In [36]:
training_data_accuracy = accuracy_score(y_train,x_train_prediction)

In [37]:
print("Accuracy training score :",training_data_accuracy)

Accuracy training score : 0.9309375


In [38]:
x_test_prediction = rating_model.predict(x_test)
test_data_accuracy = accuracy_score(y_test,x_test_prediction)
print("Accuracy test score :",test_data_accuracy)

Accuracy test score : 0.865


In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test, x_test_prediction))


              precision    recall  f1-score   support

           1       0.88      0.85      0.86       400
           5       0.85      0.88      0.87       400

    accuracy                           0.86       800
   macro avg       0.87      0.86      0.86       800
weighted avg       0.87      0.86      0.86       800



In [40]:
import pickle

In [41]:
import os
print(os.getcwd())


C:\Users\sgaui\Desktop\new\sentimental


In [42]:
filename = "trained_model.sav"
pickle.dump(rating_model, open(filename,"wb"))

In [43]:
# to load the model use "loaded_model = pickle.load(open("trained_model.sav","rb"))"


In [44]:
review = "The movie is too good. It is amazing"

In [ ]:

user_review = input("Enter movie review: ")


user_review = [stemming(user_review)]


review_vector = vectorizer.transform(user_review)


prediction = rating_model.predict(review_vector)


if prediction[0] == 5:
    print("Positive Review")
else:
    print("Negative Review")
